In [1]:
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import urllib
import numpy as np
import os
import time
from tqdm import tqdm
from selenium import webdriver
%matplotlib inline

In [2]:
df = pd.read_csv('/Users/shayneufeld/Dropbox/insight/pricecamp/data/sales.csv',index_col=0)
df = df.drop_duplicates()
print('There are %.0f unique records in the database now' % df.shape[0])
print('Total revenue: %.02f' % df.amount_paid_usd.sum())

There are 115586 unique records in the database now
Total revenue: 867360.27


# define function for opening more albums

In [5]:
def reveal_albums(selenium_driver):    
    #try implementing same thing for albums
    showMore = selenium_driver.find_elements_by_css_selector('.showMore')
    #there are 2 possible showMore buttons - for shows and for albums
    #if there is a shows one, it comes first
    el = []
    if len(showMore) == 1:
        el = showMore[0]
    elif len(showMore) == 2:
        el = showMore[1]
    
    #if there is a show more button, make sure it's 'more albums'
    if el:
        title = browser.title
        text = el.find_element_by_tag_name('a').text
        while text == 'more releases...':
            el.click()
            
            if browser.title == title:
                text = el.find_element_by_tag_name('a').text
            else:
                text = browser.title
                
    
    return selenium_driver

In [6]:
def get_albums_urls(browser,page_type='album'):
    
    elems = []
    if page_type == 'album':
        elems = browser.find_elements_by_css_selector('.trackTitle')
    elif page_type == 'music':
        elems = browser.find_elements_by_css_selector('.music-grid-item')
        
    album_names,album_urls = [],[]

    for elem in elems[1:]:
        album_names.append(elem.text)
        album_urls.append(elem.find_element_by_tag_name('a').get_attribute('href'))
        #print(elem.text)
        #print(elem.find_element_by_tag_name('a').get_attribute('href'))

    browser.close()
    
    return album_names,album_urls

# get artists to scrap


In [14]:
urls_to_scrape_df = pd.read_csv('/Users/shayneufeld/Dropbox/insight/pricecamp/urls_to_scrape.csv')

In [15]:
artist_names = []
for url in urls_to_scrape_df.urls.values:
    artist_name = df[df.url==url].head(1).artist_name.values
    
    if not artist_name:
        artist_name = df[df.url==url[6:]].head(1).artist_name.values

    if type(artist_name) is not str:
        artist_name = artist_name[0]
        
    artist_names.append(artist_name)
    

In [16]:
urls_to_scrape_df['artist_names'] = artist_names

In [17]:
urls_to_scrape = urls_to_scrape_df.urls.values

# the scraper

In [18]:
cd '/Users/shayneufeld/github/pricecamp/scrapers/bandcamp_spider'

/Users/shayneufeld/GitHub/pricecamp/scrapers/bandcamp_spider


In [23]:
error_urls = []

In [30]:
artist_name

'In Love With A Ghost'

In [28]:
np.where(artist_name in artist_names)

(array([0]),)

In [29]:
artist_names

['Garoad',
 'Boy Harsher',
 'HALLEY LABS',
 'Bucketheadland',
 'Dirty Grave',
 'Wolves in the Throne Room',
 'Jay Som',
 'LORN',
 'The Dandy Warhols',
 'In Love With A Ghost',
 'FORCES',
 'micronism',
 'Film Junk',
 'Mystery Skulls',
 'Elder',
 'DVNE',
 'Dream Machine',
 'Mega Drive',
 'Keosz',
 'Knxwledge.',
 'Jute Gyte',
 'Brock Berrigan',
 'Danny Baranowsky',
 'Scandroid',
 'Ratboys',
 'Haunt',
 'Jake Kaufman',
 'Anomalie',
 'Miracle Of Sound',
 'Cleveland',
 'Deep Dark & Dangerous',
 'Docetism',
 'Stellardrone',
 'Integrity',
 'Cytotoxin',
 'All India Radio',
 'Mounika.',
 'Anderson .Paak',
 'Jakub Zytecki',
 '2814',
 'HOME',
 '猫 シ Corp.',
 'King Gizzard & The Lizard Wizard',
 'Arch Echo',
 'MIKE',
 'Mønic',
 'DANCE WITH THE DEAD',
 'SHEER MAG',
 'Welcome to Night Vale',
 'Roosevelt',
 'Youngblood Supercult',
 'Papadosio',
 'LIL UGLY MANE',
 'Randal Collier-Ford',
 'Griffin McElroy',
 'soccer mommy',
 'Trevor Something',
 'Schwefelgelb',
 'ALEX & Tokyo Rose',
 'FM-84',
 'Less Art',

In [44]:
artist_names[10]

'FORCES'

In [34]:
urls_to_scrape[0]

'//garoad.bandcamp.com/album/va-11-hall-a-second-round'

In [ ]:
for artist_url,artist_name in zip(urls_to_scrape[10:],artist_names[10:]):
    '''
    get input parameters
    '''
    if artist_url[0] == 'h':
        curr_url = artist_url
    elif artist_url[0] == '/':
        curr_url = 'https:' + artist_url
    else:
        print('URL BROKEN')
        
    #get list of albums and urls to go through for this artist
    browser = webdriver.Firefox()
    browser.get(curr_url)
    browser_title = browser.title

    #try to clicking throuhg more albums
    browser = reveal_albums(browser)
    
    #if it went to the music page, the title should have changed
    DO_NOT_SCRAPE = False
    if browser.title == browser_title: #then we are on the same page
        album_names,album_urls = get_albums_urls(browser,page_type='album')
    elif 'Music' in browser.title:
        album_names,album_urls = get_albums_urls(browser,page_type='music')
    else:
        DO_NOT_SCRAPE = True
        print('ERROR - page not album or music type: %s' % artist_url)
    
    if not DO_NOT_SCRAPE:
        if album_urls:
            for url in album_urls:
                os.system("scrapy crawl bandcamp -a start_urls='%s'" % (str(url)))
        else:
            print('ERROR - no album urls: %s' % artist_url)
            error_urls.append(artist_url)

ERROR - page not album or music type: //filmjunk.bandcamp.com/album/david-lynch
ERROR - no album urls: //catsystemcorp.bandcamp.com
ERROR - page not album or music type: //liluglymane.bandcamp.com/album/third-side-of-tape
ERROR - no album urls: //desired.bandcamp.com
ERROR - no album urls: //ztapes.bandcamp.com
ERROR - no album urls: //birocratic.bandcamp.com
ERROR - no album urls: //silvahound.bandcamp.com


In [1]:
artist_url

NameError: name 'artist_url' is not defined

In [99]:
artist_url

'https://songs-of-arrakis.bandcamp.com/album/aurora-majesty'

In [7]:
artist_url = 'https://cryochamber.bandcamp.com/album/deus-sive-natura'

'''
get input parameters
'''
if artist_url[0] == 'h':
    curr_url = artist_url
elif artist_url[0] == '/':
    curr_url = 'https:' + artist_url
else:
    print('URL BROKEN')

#get list of albums and urls to go through for this artist
browser = webdriver.Firefox()
browser.get(curr_url)
browser_title = browser.title

#try to clicking throuhg more albums
browser = reveal_albums(browser)

In [8]:
#if it went to the music page, the title should have changed
if browser.title == browser_title: #then we are on the same page
    album_names,album_urls = get_albums_urls(browser,page_type='album')
elif 'Music' in browser.title:
    album_names,album_urls = get_albums_urls(browser,page_type='music')
else:
    DO_NOT_SCRAPE = True
    print('ERROR - page not album or music type: %s' % artist_url)

In [9]:
album_urls[10:12]

['https://cryochamber.bandcamp.com/album/legends-of-the-wood',
 'https://cryochamber.bandcamp.com/album/abduction']

In [12]:
#if not DO_NOT_SCRAPE:
if album_urls:
    for url in album_urls[10:12]:
        os.system("scrapy crawl bandcamp -a start_urls='%s'" % (str(url)))
else:
    print('ERROR - no album urls: %s' % artist_url)
    error_urls.append(artist_url)

In [151]:
artist_name = 'Middle Kids'

In [231]:
'''
run bandcamp spider
'''
cd '/Users/shayneufeld/github/pricecamp/scrapers/bandcamp_spider'

album_url = album_urls
artist_name = 'Middle Kids'

for url in album_urls:
    os.system("scrapy crawl bandcamp -a start_urls='%s' -a artist_name='%s'" % (str(url),artist_name))

In [216]:
test = str(album_url[1:])
test[1]

"'"

In [217]:
test = test.replace("[",'')
test = test.replace("]",'')
test = test.replace("'",'')
test = test.replace(" ",'')
test = test.split(',')

In [221]:
test[1]

'https://middlekids.bandcamp.com/track/egde-of-town'

In [203]:
test[0][1]

"'"

In [63]:
np.savetxt('/Users/shayneufeld/Dropbox/insight/pricecamp/urls_scraped',list(urls_scraped))

TypeError: Mismatch between array dtype ('<U157') and format specifier ('%.18e')

In [29]:
file_names = os.listdir(folder)

In [26]:
file_name

'/Users/shayneufeld/dropbox/insight/pricecamp/data/albums_fixed/that-positive-feeling-music-license-Alumo.csv'

In [32]:
file_name = album_name + '-' + artist_name + '.csv'
file_name

'that-positive-feeling-music-license-Alumo.csv'

In [33]:
file_name in file_names

True

In [ ]:
artist_name = df[df.url==url[6:]].head(1).artist_name.values
artist_name

In [31]:
file_names[100]

'adrift-in-sleepwakefulness-[Bucketheadland].csv'

In [ ]:
type(artist_name) is not str

In [ ]:
urls_scraped = []
folder =  '/Users/shayneufeld/Dropbox/insight/pricecamp/data/albums_fixed/'
files = os.listdir(folder)

for i,file in enumerate(files):
    try:
        if not file[0] == '.':
            df = pd.read_csv(os.path.join(folder,file))
            url = df.album_url.values[0]
            urls_scraped.append(url)
    except:
        pass

In [ ]:
urls_scraped

In [ ]:
d = {'url':urls_scraped}
urls_scraped_df = pd.DataFrame(data = d)
urls_scraped_df.to_csv('/Users/shayneufeld/Dropbox/insight/pricecamp/data/urls_scraped.csv')